# CEE6501 — Lecture 3.2

## The Direct Stiffness Method (DSM) for Trusses (PART 1)

## Learning Objectives

By the end of this lecture, you will be able to:

- Construct the **local-to-global transformation** for a truss member
- Compute element stiffness in global coordinates:
  $$
  \boldsymbol{k} = \boldsymbol{T}^{\mathsf{T}} \boldsymbol{k}' \boldsymbol{T}
  $$
- Assemble the **global stiffness matrix** $\boldsymbol{K}$ using scatter-add
- Explain why an **unsupported structure** leads to a **singular stiffness matrix**
- Apply boundary conditions through **DOF partitioning** and solve for displacements
- Recover **member axial forces** from global displacements for design


## Agenda

1. Local-to-global transformation using direction cosines  
2. Member stiffness relations in the global coordinate system  
3. Assembly of the global stiffness matrix $\boldsymbol{K}$  (manual)

Big idea:
- A truss is a network of axial springs.
- Each element contributes stiffness to shared DOFs.
- Assembly is adding contributions into the right global rows/columns.


## Part 1 — Local to Global Transformation

### Generic Truss Element in a Structure

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.55;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_TrussGlobal.png" style="width:100%; height:auto;">
    </figure>
  </div>
  <div style="flex:0.45; font-size:0.95em;">
    <ul>
      <li>A truss member is embedded in a <strong>global coordinate system</strong> $(X,Y)$</li>
      <li>The element stiffness was derived in a <strong>local coordinate system</strong> $(x,y)$ aligned with the member</li>
      <li>The member orientation is defined by an angle $\theta$, measured <strong>counterclockwise</strong> from global $X$ to local $x$</li>
      <li>Structural assembly requires <strong>transforming forces and displacements</strong> between local and global coordinates</li>
    </ul>
  </div>
</div>

### Local/Global Perspective

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.95;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_Local2Global.png" style="width:80%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;">
        <!-- Local (left) vs. global (right) descriptions of the same member -->
      </figcaption>
    </figure>
  </div>
</div>

- **Local coordinate system (left):** forces $\boldsymbol{Q}$, displacements $\boldsymbol{u}$
- **Global coordinate system (right):** forces $\boldsymbol{F}$, displacements $\boldsymbol{v}$

### Global ($\boldsymbol{F}$) → Local ($\boldsymbol{Q}$) Force Transformation: Trigonometry

<div style="display:flex; gap:1.5rem; align-items:center;">

  <!-- Figures (left, stacked) -->
  <div style="flex:0.45; display:flex; flex-direction:column; gap:0.15rem;">
    <figure style="margin:0; text-align:center;">
      <img src="assets/L2_Local2Global_Member.png" style="width:100%; height:auto;">
    </figure>
    <figure style="margin:0; text-align:center;">
      <img src="assets/L2_Local2Global_Trig.png" style="width:100%; height:auto;">
    </figure>
  </div>

  <!-- Bullets with equations (right) -->
  <div style="flex:0.55; font-size:0.95em;">
    <ul>
      <strong>At node $b$ (start node):</strong>
      <ul>
        <li>$$Q_1 = F_1\cos\theta + F_2\sin\theta$$</li>
        <li>$$Q_2 = -F_1\sin\theta + F_2\cos\theta$$</li>
      </ul>
      <strong>At node $e$ (end node):</strong>
      <ul>
        <li>$$Q_3 = F_3\cos\theta + F_4\sin\theta$$</li>
        <li>$$Q_4 = -F_3\sin\theta + F_4\cos\theta$$</li>
      </ul>
    </ul>
  </div>

</div>


### Global ($\boldsymbol{F}$) → Local ($\boldsymbol{Q}$) Force Transformation: Matrix Form

- Local member forces $\boldsymbol{Q}$ are obtained by **rotating** global nodal forces $\boldsymbol{F}$ into the member’s local coordinate system
- Each $2\times2$ block applies a **rotation by $\theta$** at a node
- The transformation changes **direction only**, not force magnitude
- This operation is a **pure coordinate transformation**

$$
\begin{Bmatrix}Q_1\\Q_2\\Q_3\\Q_4\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
}_{\boldsymbol{T}}
\begin{Bmatrix}F_1\\F_2\\F_3\\F_4\end{Bmatrix}
$$

$$\boldsymbol{Q} = \boldsymbol{T}\boldsymbol{F}$$

### Direction Cosines: Rotation Terms

- Direction cosines define the **orientation of a truss member** in the global $(X,Y)$ coordinate system
- The angle $\theta$ is measured **counterclockwise** from the global $X$ axis to the local $x$ axis
- Computed directly from the **nodal coordinates** of the element (b = start node, e = end node)

$$
\cos\theta = \frac{X_e - X_b}{\sqrt{(X_e-X_b)^2 + (Y_e-Y_b)^2}},
\qquad
\sin\theta = \frac{Y_e - Y_b}{\sqrt{(X_e-X_b)^2 + (Y_e-Y_b)^2}}
$$

- The denominator is the **member length** $L$
- Once computed, $\cos\theta$ and $\sin\theta$ are **reused throughout the element formulation**


### Global ($\boldsymbol{v}$) → Local ($\boldsymbol{u}$) Displacements

- Nodal displacements are transformed using the **same rotation matrix** as forces
- Displacements and forces transform identically because they are defined along the **same directions**
- This is a **pure coordinate rotation**, not a change in deformation

$$
\boldsymbol{u} = \boldsymbol{T}\boldsymbol{v}
$$

- $\boldsymbol{v}$: global displacement vector
- $\boldsymbol{u}$: local displacement vector

### Local ($\boldsymbol{Q}$) → Global ($\boldsymbol{F}$) Force: Transformation Equations

- This is the **reverse of the global → local process**
- Local member forces are **rotated back** into the global $(X,Y)$ directions

**At node $b$ (start node):**
$$
F_1 = Q_1\cos\theta - Q_2\sin\theta,
\qquad
F_2 = Q_1\sin\theta + Q_2\cos\theta
$$

**At node $e$ (end node):**
$$
F_3 = Q_3\cos\theta - Q_4\sin\theta,
\qquad
F_4 = Q_3\sin\theta + Q_4\cos\theta
$$

### Local ($\boldsymbol{Q}$) → Global ($\boldsymbol{F}$) Force: Matrix Form

- Local member forces are mapped to global nodal forces using the **transpose** of the global→local transformation

$$
\small{
\begin{Bmatrix}
F_1\\ F_2\\ F_3\\ F_4
\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
\cos\theta & -\sin\theta & 0 & 0\\
\sin\theta & \cos\theta  & 0 & 0\\
0 & 0 & \cos\theta & -\sin\theta\\
0 & 0 & \sin\theta & \cos\theta
\end{bmatrix}
}_{\boldsymbol{T}^{\mathsf{T}}}
\begin{Bmatrix}
Q_1\\ Q_2\\ Q_3\\ Q_4
\end{Bmatrix}
}
$$

$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}} \boldsymbol{Q}
$$

**Recall (Global → Local):**
$$
\boldsymbol{T}
=
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
$$


### Properties of the Transformation Matrix

$$\boldsymbol{T}^{-1} = \boldsymbol{T}^T$$

The transformation matrix is **orthogonal**, which greatly simplifies operations involving the stiffness transformations.

### Summary — Local ↔ Global Transformations

**Direction cosines (member orientation):**
$$
\cos\theta = \frac{X_e-X_b}{L}, \qquad \sin\theta = \frac{Y_e-Y_b}{L}
$$

**Global → Local (forces or displacements):**
$$
\boldsymbol{Q} = \boldsymbol{T}\boldsymbol{F},
\qquad
\boldsymbol{u} = \boldsymbol{T}\boldsymbol{v}
$$

**Local → Global (forces or displacements):**
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{Q},
\qquad
\boldsymbol{v} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{u}
$$

$$
\small{
\boldsymbol{T}
=
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
}
$$

- $\boldsymbol{T}$ is a **pure rotation matrix** (no translation)
- $\boldsymbol{T}^{-1} = \boldsymbol{T}^{\mathsf{T}}$ (orthogonal)


## Part 2 — Member Stiffness in the Global Coordinate System

### Goal

- We have derived the **local stiffness relation** (Lecture 3.1 today):
  $$\boldsymbol{Q} = \boldsymbol{k}\boldsymbol{u}$$
- We also know how to **transform forces and displacements** between local and global systems
- Objective: express the **member stiffness relation entirely in global coordinates**

### Transformation Chain — From Local to Global (Step-by-Step)

**Step 1 — Local force–displacement relation**
$$
\boldsymbol{Q} = \boldsymbol{k}\,\boldsymbol{u}
$$
The element stiffness matrix $\boldsymbol{k}$ relates **local nodal displacements** $\boldsymbol{u}$ to the corresponding **local nodal forces** $\boldsymbol{Q}$.

**Step 2 — Transform local forces to global forces**
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{Q}
$$
Global nodal forces are obtained by rotating the local force vector into the global coordinate system.

Substitute the local stiffness relation from Step 1, $\boldsymbol{Q} = \boldsymbol{k}\,\boldsymbol{u}$, into the force transformation:
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{Q}
\;\;\Longrightarrow\;\;
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\bigl(\boldsymbol{k}\,\boldsymbol{u}\bigr)
$$


**Step 3 — Transform global displacements to local displacements**
$$
\boldsymbol{u} = \boldsymbol{T}\,\boldsymbol{v}
$$

Substitute the displacement transformation into the previous expression:
$$
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{k}\,\boldsymbol{u}
\;\;\Longrightarrow\;\;
\boldsymbol{F} = \boldsymbol{T}^{\mathsf{T}}\boldsymbol{k}\bigl(\boldsymbol{T}\,\boldsymbol{v}\bigr)
$$


**Step 4 — Rearrange into global stiffness form**
$$
\boldsymbol{F} = (\boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{k}\,\boldsymbol{T})\,\boldsymbol{v}
$$




**Step 5 — Final global stiffness relation**

Define the global element stiffness matrix:
$$
\boldsymbol{K} = \boldsymbol{T}^{\mathsf{T}}\,\boldsymbol{k}\,\boldsymbol{T}
$$

This is the element stiffness relation used directly in global assembly for the Direct Stiffness Method.

textbook notation:
$$
\boxed{\boldsymbol{F} = \boldsymbol{K}\,\boldsymbol{v}}
$$

our notation (interchangeable):
$$
\boxed{\boldsymbol{f} = \boldsymbol{K}\,\boldsymbol{u}}
$$


### Calculating the Global Stiffness Matrix, $\boldsymbol{K}$

The global element stiffness matrix is obtained by **rotating the local axial stiffness**
into the global coordinate system:

<br/>

$$
{\scriptsize
\boldsymbol{K}
=
\begin{bmatrix}
\cos\theta & -\sin\theta & 0 & 0\\
\sin\theta & \cos\theta  & 0 & 0\\
0 & 0 & \cos\theta & -\sin\theta\\
0 & 0 & \sin\theta & \cos\theta
\end{bmatrix}
\cdot
\frac{EA}{L}
\begin{bmatrix}
1 & 0 & -1 & 0\\
0 & 0 & 0 & 0\\
-1 & 0 & 1 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
\cdot
\begin{bmatrix}
\cos\theta & \sin\theta & 0 & 0\\
-\sin\theta & \cos\theta & 0 & 0\\
0 & 0 & \cos\theta & \sin\theta\\
0 & 0 & -\sin\theta & \cos\theta
\end{bmatrix}
}
$$

<br/>

Carrying out the matrix multiplication yields the **closed-form global stiffness matrix**:

$$
\boldsymbol{K} = \frac{EA}{L}
\begin{bmatrix}
\cos^2\theta & \cos\theta\sin\theta & -\cos^2\theta & -\cos\theta\sin\theta \\
\cos\theta\sin\theta & \sin^2\theta & -\cos\theta\sin\theta & -\sin^2\theta \\
-\cos^2\theta & -\cos\theta\sin\theta & \cos^2\theta & \cos\theta\sin\theta \\
-\cos\theta\sin\theta & -\sin^2\theta & \cos\theta\sin\theta & \sin^2\theta
\end{bmatrix}
$$



### Key Observations — Global Stiffness Matrix, $\boldsymbol{K}$

- The member global stiffness matrix $\boldsymbol{K}$ is **symmetric**, just like the local stiffness matrix  
- $\boldsymbol{K}$ represents the same physical behavior, but expressed in the **global $(X,Y)$ coordinate system**
- Each coefficient $K_{ij}$ is the **force at global DOF $i$** required to produce a **unit displacement at global DOF $j$**, with all other displacements fixed


### Direct Calculation of Global Stiffness Matrix, $\boldsymbol{K}$

- One *could* derive $\boldsymbol{K}$ directly by:
  - Applying **unit global displacements** to a generic inclined truss member
  - Evaluating the **global end forces** required to produce each unit displacement in global coordinates
- The **$j$th column of $\boldsymbol{K}$** gives the global nodal force pattern caused by $v_j=1$

- This approach is **theoretically equivalent** to the transformation-based derivation and provides a clear physical interpretation of $\boldsymbol{K}$.

- However, it is **significantly more labor-intensive**, and is mainly useful as a **verification tool**, rather than for routine analysis.


### Example: First Column of $\boldsymbol{K}$

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.85;">
    <figure style="margin:0; text-align:center; display:flex; flex-direction:column;">
      <img src="assets/L2_GlobalDerive1.png" style="width:80%; height:auto;">
      <figcaption style="font-size:0.75em; margin-top:0.4em;">
        <!-- Local (left) vs. global (right) descriptions of the same member -->
      </figcaption>
    </figure>
  </div>
</div>

- Impose $v_1 = 1$, all other global displacements zero
- Project the resulting axial deformation onto the member axis
- Resolve the axial force back into global components

You recover the **first column of $\boldsymbol{K}$**, which should exactly match $\boldsymbol{T}^\mathsf{T}\boldsymbol{k}\boldsymbol{T}$.

## Part 3 — Assembling the Global Structure Stiffness Matrix
>Manual Method

### Example Structure

We now move from **element-level** to a **complete truss structure** behavior.

- Structure composed of **3 axial truss elements**
- **4 nodes**, each with $(X,Y)$ translational DOFs, node 1 at the top
- Total system size: **8 global degrees of freedom**
  - node 1: DOF (1,2)
  - node 2: DOF (3,4)
  - node 3: DOF (5,6)
  - node 4: DOF (7,8)

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.5;">
    <img src="assets/L2_Structure1.png" style="width:80%;">
  </div>
  <div style="flex:0.5;">
    <img src="assets/L2_Structure2.png" style="width:80%;">
  </div>
</div>

### Element-Level View of the Structure

<div style="text-align:center;">
  <img src="assets/L2_Structure3.png" style="width:80%;">
</div>

- Nodes **2, 3, and 4** are **pinned** (no displacement)
- **Node 1** is free to move
- All three elements are connected at **node 1**
- A displacement at node 1 induces forces in **all connected members**

### Element Forces and Notation

Local forces are defined **per element**:

- Superscript $(e)$ → element number
- Subscript $(i)$ → local DOF index

Examples:
- $F^{(1)}_3$ → force at local DOF 3 in **element 1**
- $F^{(2)}_1$ → force at local DOF 1 in **element 2**



### Equilibrium Equations at Node 1

Because all elements share **node 1**, equilibrium at this node couples the response of all members.

$P$ = external applied loaded

$F$ = internal forces

<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.55;">
    <img src="assets/L2_Structure4.png" style="width:80%;">
  </div>
  <div style="flex:0.45; font-size:0.95em;">
    <p><strong>Force equilibrium at node 1:</strong></p>
    $$P_1 = F_3^{(1)} + F_1^{(2)} + F_3^{(3)}$$
    $$P_2 = F_4^{(1)} + F_2^{(2)} + F_4^{(3)}$$
  </div>
</div>

### Compatability Equations

<div style="text-align:center;">
  <img src="assets/L2_Structure3.png" style="width:70%;">
</div>

$d_1$ and $d_2$ are global displacements in X and Y at node 1

$$
\begin{aligned}
\text{Member }(1): \quad
& v^{(1)}_1 = v^{(1)}_2 = 0,
&& v^{(1)}_3 = d_1,
&& v^{(1)}_4 = d_2 \\[6pt]
\text{Member }(2): \quad
& v^{(2)}_1 = d_1,
&& v^{(2)}_2 = d_2,
&& v^{(2)}_3 = v^{(2)}_4 = 0 \\[6pt]
\text{Member }(3): \quad
& v^{(3)}_1 = v^{(3)}_2 = 0,
&& v^{(3)}_3 = d_1,
&& v^{(3)}_4 = d_2
\end{aligned}
$$


### Member 1 — Force–Displacement Relations

Compatibility (Member 1): $v^{(1)}_1 = v^{(1)}_2 = 0,\; v^{(1)}_3 = d_1,\; v^{(1)}_4 = d_2$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(1)}_1\\
F^{(1)}_2\\
F^{(1)}_3\\
F^{(1)}_4
\end{Bmatrix}
=
\begin{bmatrix}
K^{(1)}_{11} & K^{(1)}_{12} & K^{(1)}_{13} & K^{(1)}_{14}\\
K^{(1)}_{21} & K^{(1)}_{22} & K^{(1)}_{23} & K^{(1)}_{24}\\
K^{(1)}_{31} & K^{(1)}_{32} & \boxed{K^{(1)}_{33}} & \boxed{K^{(1)}_{34}}\\
K^{(1)}_{41} & K^{(1)}_{42} & \boxed{K^{(1)}_{43}} & \boxed{K^{(1)}_{44}}
\end{bmatrix}
\begin{Bmatrix}
v^{(1)}_1 = 0\\
v^{(1)}_2 = 0\\
v^{(1)}_3 = d_1\\
v^{(1)}_4 = d_2
\end{Bmatrix}
}
$$

The forces acting at **end node of member 1** (Global node 1) are:

$$
F^{(1)}_3
=
K^{(1)}_{33} d_1
+
K^{(1)}_{34} d_2
$$

$$
F^{(1)}_4
=
K^{(1)}_{43} d_1
+
K^{(1)}_{44} d_2
$$


### Member 2 — Force–Displacement Relations

Compatibility (Member 2): $v^{(2)}_1 = d_1,\; v^{(2)}_2 = d_2,\; v^{(2)}_3 = v^{(2)}_4 = 0$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(2)}_1\\
F^{(2)}_2\\
F^{(2)}_3\\
F^{(2)}_4
\end{Bmatrix}
=
\begin{bmatrix}
\boxed{K^{(2)}_{11}} & \boxed{K^{(2)}_{12}} & K^{(2)}_{13} & K^{(2)}_{14}\\
\boxed{K^{(2)}_{21}} & \boxed{K^{(2)}_{22}} & K^{(2)}_{23} & K^{(2)}_{24}\\
K^{(2)}_{31} & K^{(2)}_{32} & K^{(2)}_{33} & K^{(2)}_{34}\\
K^{(2)}_{41} & K^{(2)}_{42} & K^{(2)}_{43} & K^{(2)}_{44}
\end{bmatrix}
\begin{Bmatrix}
v^{(2)}_1 = d_1\\
v^{(2)}_2 = d_2\\
v^{(2)}_3 = 0\\
v^{(2)}_4 = 0
\end{Bmatrix}
}
$$

The forces acting at **start node of member 2** (Global node 1) are:

$$
F^{(2)}_1
=
K^{(2)}_{11} d_1
+
K^{(2)}_{12} d_2
$$

$$
F^{(2)}_2
=
K^{(2)}_{21} d_1
+
K^{(2)}_{22} d_2
$$


### Member 3 — Force–Displacement Relations

Compatibility (Member 3): $v^{(3)}_1 = v^{(3)}_2 = 0,\; v^{(3)}_3 = d_1,\; v^{(3)}_4 = d_2$

Element-level global stiffness expression:

$$
{\small
\begin{Bmatrix}
F^{(3)}_1\\
F^{(3)}_2\\
F^{(3)}_3\\
F^{(3)}_4
\end{Bmatrix}
=
\begin{bmatrix}
K^{(3)}_{11} & K^{(3)}_{12} & K^{(3)}_{13} & K^{(3)}_{14}\\
K^{(3)}_{21} & K^{(3)}_{22} & K^{(3)}_{23} & K^{(3)}_{24}\\
K^{(3)}_{31} & K^{(3)}_{32} & \boxed{K^{(3)}_{33}} & \boxed{K^{(3)}_{34}}\\
K^{(3)}_{41} & K^{(3)}_{42} & \boxed{K^{(3)}_{43}} & \boxed{K^{(3)}_{44}}
\end{bmatrix}
\begin{Bmatrix}
v^{(3)}_1 = 0\\
v^{(3)}_2 = 0\\
v^{(3)}_3 = d_1\\
v^{(3)}_4 = d_2
\end{Bmatrix}
}
$$

The forces acting at **end node of member 3** (Global node 1) are:

$$
F^{(3)}_3
=
K^{(3)}_{33} d_1
+
K^{(3)}_{34} d_2
$$

$$
F^{(3)}_4
=
K^{(3)}_{43} d_1
+
K^{(3)}_{44} d_2
$$

### Global Stiffness Matrix for Free DOFs

$$
\begin{aligned}
P_1 &= F_3^{(1)} + F_1^{(2)} + F_3^{(3)} \\
P_2 &= F_4^{(1)} + F_2^{(2)} + F_4^{(3)}
\end{aligned}
$$

Substituting element-level force expressions (from last 3 slides) into the node 1 force equilibrium gives:
$$
\begin{aligned}
P_1 &=
\bigl(K_{33}^{(1)} + K_{11}^{(2)} + K_{33}^{(3)}\bigr) d_1
+
\bigl(K_{34}^{(1)} + K_{12}^{(2)} + K_{34}^{(3)}\bigr) d_2 \\[6pt]
P_2 &=
\bigl(K_{43}^{(1)} + K_{21}^{(2)} + K_{43}^{(3)}\bigr) d_1
+
\bigl(K_{44}^{(1)} + K_{22}^{(2)} + K_{44}^{(3)}\bigr) d_2
\end{aligned}
$$

These equations can be written compactly as:
$$
\begin{Bmatrix}
P_1 \\ P_2
\end{Bmatrix}
=
\underbrace{
\begin{bmatrix}
K_{33}^{(1)} + K_{11}^{(2)} + K_{33}^{(3)} &
K_{34}^{(1)} + K_{12}^{(2)} + K_{34}^{(3)} \\[4pt]
K_{43}^{(1)} + K_{21}^{(2)} + K_{43}^{(3)} &
K_{44}^{(1)} + K_{22}^{(2)} + K_{44}^{(3)}
\end{bmatrix}
}_{\boldsymbol{K}_{\text{s}}}
\begin{Bmatrix}
d_1 \\ d_2
\end{Bmatrix}
$$

$$
\boxed{\;\boldsymbol{P} = \boldsymbol{K}_{\text{s}}\,\boldsymbol{d}\;}
$$

### Physical Interpretation of the Structure Stiffness Matrix

- The **structure stiffness matrix** $\boldsymbol{K}_s$ has the same physical meaning as an element stiffness matrix, but at the **structure (joint) level**
- A stiffness coefficient $K_{s,ij}$ represents:
  - The **joint force** at DOF $i$
  - Required to cause a **unit displacement** at DOF $j$
  - While **all other joint displacements are zero**
- Equivalently:
  - Each **column** of $\boldsymbol{K}_s$ corresponds to a unit displacement pattern
  - The column entries are the **resulting joint forces** needed to enforce that displacement


### Example — First Column of $\boldsymbol{K}_s$

- To obtain the **first column** of $\boldsymbol{K}_s$:
  - Impose a **unit displacement** at the first free DOF:
    $$
    d_1 = 1, \qquad d_2 = 0
    $$
  - All other joint displacements are held fixed
- The resulting joint forces (DOF 1, 2) define the **first column** of $\boldsymbol{K}_s$


<div style="display:flex; gap:1.5rem; align-items:center;">
  <div style="flex:0.5;">
    <img src="assets/L2_GlobalManual1.png" style="width:90%;">
  </div>
  <div style="flex:0.5;">
    <img src="assets/L2_GlobalManual2.png" style="width:90%;">
  </div>
</div>


## Wrap-Up

Today you started building the DSM pipeline for trusses:
- local bar stiffness $\to$ transformation $\to$ global element stiffness $\to$ assembling global stiffness matrix for a structure

Next Lecture: continue building the DSM pipeline for trusses and implement DSM in Python for a worked truss example and discuss efficiency (sparsity/bandedness).
